# Dados de Mercado

In [1]:
#Importando pacotes
import pandas as pd
import numpy as np
from urllib.request import urlopen
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import mstats
warnings.filterwarnings("ignore")

In [2]:
#Descriminando URL de download
url = "https://api.data.economatica.com/1/oficial/datafeed/download/1/Fun%2FTYUAobeTU5%2B7Sr6sS69qytB6QnXDiBqSSXsMd3%2Fg%2FvCMocCW3%2FSWloeuPGhqv%2FY5xqKATqtEpp81AkPRytV9L3FuX7947p2md8jic51QGq6%2FZAeBTJSsftlcSEdus5Q%2F5lF7xQAKVtVGEOE8sWI9J36UHSDm7iQ2PG%2BUM%2FMWk2E2ehX6IizBips6zCT0qfj3fD%2BAieWhNX72DfXwqOUADq9kOdrVZ8%2Fc2d6kjSRkppj96VlpVKxAGbGTgyfpRz1XKJEIUFWEVEk2mEhONOjOzJLfWDBs3P88EZTmbSPgaqQ8tr07E3EHNRp9a9ycWuaHfQkKN1ILVpyHnYF1ng%3D%3D"
indices = "https://api.data.economatica.com/1/oficial/datafeed/download/1/jIFbIzZ6mj3itlfxKk9y5RW6yi7UT2OuKsIfrip3AbmQn3hIqrNgLKl0NmC3dkb5S4JISEgAD2dNFmrBs7bH%2BSOq4iCd%2FRwDRm9p5x7rtnWyUFAzJJJBQmUMeM48%2BzvYDrkHNHFEkvWGM4ifq%2BJ48jfgvk7dn%2BVGUKkcZxI4FjA4yXM%2FoFjWVpwgeOLBLs%2FpCuttUBMntHZfSWm%2BQv2BR9t%2FwlQPJyvEyvzldFtsC6ugE%2B4EOxGywjw1%2B037t2j0UxZTHoJr1Z4f%2FCXO%2BXGUi1FP7xepIiAmnQLxs3bS8uR9lcTWeRQh9P6EbuqfEY8OC1lz%2B0kAgwoRe65rWMZYHQ%3D%3D"

In [4]:
##%%time
#Abrindo a base
data = pd.read_csv(urlopen(url),encoding='latin-1')
#Retirando sufixo
data['Ativo']=data['Ativo'].str.rstrip('<XBSP>')
#Declarando coluna data como datatime
data['Data'] = pd.to_datetime(data['Data'])
#Renomeando colunas
data.columns = ['ativo','data',\
                "pl12m","pl3m","plex",\
                "pvp",\
                "prec12m","prec3m","precex",\
                "pebitda12m","pebitda3m","pebitdaex",\
                "pfcf12m","pfcf3m","pfcfex",\
                "lp12m","lp3m","lpex",\
                "marketcap",\
                "ev",\
                "estrutcapvm",\
                "evebitdaemp12m","evebitdaemp3m","evebitdaempex",\
                "evebitdacls12m","evebitdacls3m","evebitdaclsex",\
                "evebitemp12m","evebitemp3m","evebitempex",\
                "evebitcls12m","evebitcls3m","evebitclsex",\
                "evvd12m","evvd3m","evvdex"]

In [5]:
#Confirmando o número de empresas
unique=data['ativo'].unique()
len(unique)

311

In [6]:
#data

In [7]:
#Declarando hifens como dados faltantes NAN
data[data == "-"] = np.nan
#Preenchendo valores ausentes (médoto para frente (ffill))
data = data.set_index('ativo', append=True)
data = data.groupby(level=1).ffill()
data = data.reset_index(level=1)

In [8]:
data=data.sort_values(['data', "ativo"], ascending=[True, True])
#data.head(5)

In [10]:
#IBOV que  ira auxiliar em construir um índice com datas do último dia do mês para o calendário brasileiro
#Abrindo a base
indice = pd.read_csv(urlopen(indices),encoding='latin-1')
#Retirando sufixo
indice['Ativo']=indice['Ativo'].str.rstrip('<XBSP>')
#Base somente com o IBOV
ibov=indice[indice['Ativo']=='IBOV'].iloc[:,:3]
#Declarando coluna data como datatime
ibov['Data'] = pd.to_datetime(ibov['Data'])
#Declarando hifens como dados faltantes NAN
ibov[ibov=='-']=np.nan
#Excluindo NAN (dias que a B3 não abriu)
ibov=ibov.dropna()
#Renomeia colunas
ibov.columns = ['ativo','data','fech']
#Identificando o último dia de cada mês
groups = ibov.groupby([ibov['data'].dt.year, ibov['data'].dt.month])['data']
last = []
for idx, group in groups:
    y = last.append(group.iloc[-1])
#last

In [11]:
#Marcando a última data disponível no objeto index e comparando com o ultimo dia do mês
import datetime
import calendar
from pandas.tseries.offsets import BMonthEnd
from pandas.tseries.offsets import BDay

today = datetime.date.today()
offset = BMonthEnd()
lastdaymt_any = offset.rollforward(today)
lastdaymt_ifdec = offset.rollforward(today) - BDay(1)

if (today.month==12):
    lastdaymt = lastdaymt_ifdec
else:
    lastdaymt = lastdaymt_any
    

#Preparando dataframe de referência onde todos ativos terão todas as datas de fim de mês
index=pd.DataFrame({'data':last}) #usando a lista definida acima#



if (index.iloc[-1][0] != lastdaymt):
    index=index[:-1] #tirando o mês que ainda não terminou
else:
    index=index
    
indexx=pd.concat([index]*len(unique), ignore_index=True)
listaativo=np.repeat(unique,len(index))
indexx['ativo']=listaativo
indexx

,data,ativo
0,2001-11-30,RRRP3
1,2001-12-28,RRRP3
2,2002-01-31,RRRP3
3,2002-02-28,RRRP3
4,2002-03-28,RRRP3
...,...,...
77745,2022-04-29,YDUQ3
77746,2022-05-31,YDUQ3
77747,2022-06-30,YDUQ3
77748,2022-07-29,YDUQ3


In [26]:
#Marcando a última data disponível no objeto index e comparando com o ultimo dia do mês
import datetime
import calendar
from pandas.tseries.offsets import BMonthEnd
from pandas.tseries.offsets import BDay

today = datetime.date.today()
offset = BMonthEnd()
lastdaymt = offset.rollforward(today)
#lastdaymt_ifdec = offset.rollforward(today) - BDay(1)

#VERIFICA SE O ULTIMO DIA DO MÊS SERÁ 31 PARA O CASO DE DEZEMBRO, E APLICA A FORMULA CORRETA (TODOS OS MESES SERÃO A MESMA, EXCETO DEZEMBRO)
if (lastdaymt.day == 31) & (lastdaymt.month ==12):
    lastdaymt = offset.rollforward(today) - BDay(1)
else:
    lastdaymt = offset.rollforward(today)


#Preparando dataframe de referência onde todos ativos terão todas as datas de fim de mês
index=pd.DataFrame({'data':last}) #usando a lista definida acima#



if (index.iloc[-1][0] != lastdaymt):
    index=index[:-1] #tirando o mês que ainda não terminou
else:
    index=index
    
indexx=pd.concat([index]*len(unique), ignore_index=True)
listaativo=np.repeat(unique,len(index))
indexx['ativo']=listaativo
indexx

2022-09-30 00:00:00


,data,ativo
0,2001-11-30,RRRP3
1,2001-12-28,RRRP3
2,2002-01-31,RRRP3
3,2002-02-28,RRRP3
4,2002-03-28,RRRP3
...,...,...
77745,2022-04-29,YDUQ3
77746,2022-05-31,YDUQ3
77747,2022-06-30,YDUQ3
77748,2022-07-29,YDUQ3


In [30]:
#Unindo a base de data/ativo completo com a base de dados (aparecerão Nan's nas datas sem valor)
mercado=pd.merge(indexx, data, how="left", on=["ativo", "data"])
#mercado

In [11]:
#Salvando em excel
#mercado.to_excel('mercado.xlsx')

In [31]:
#MATRIZ IBRX
ibrx=pd.read_excel('ibrx.xlsx', sheet_name='Sheet1', index_col='Data')
#ibrx

In [32]:
#Verificando igualdade de número de ativos
len(list(ibrx.columns))==len(unique)

True

In [33]:
#Verificando igualdade de NÚMERO de datas (o "-1" tira o mês de novembro de 2001 que tem incluído nos links mas não no IBRX)
len(list(ibrx.index))==len(index)-1

False

In [34]:
#Preparando dataframe de referência do IBRX com data e ativos empilhados
empilhados=pd.Series(ibrx.values.ravel('F'))#Empilhando colunas do dataframe
indexibrx=pd.DataFrame({'data':list(ibrx.index)}) #Criando um índice
indexxibrx=pd.concat([indexibrx]*len(list(ibrx.columns)), ignore_index=True)
listaativo=np.repeat(list(ibrx.columns),len(indexibrx))
indexxibrx['ativo']=listaativo
indexxibrx['IBRX']=empilhados

In [35]:
#Verificando igualdade entre datas do IBRX e da base DATA
list(indexibrx['data'])==list(index.loc[1:,'data'])

False

In [36]:
#Unindo base de dados com IBRX
mercado1=pd.merge(indexxibrx, mercado, how="left", on=["ativo", "data"])
mercado1

,data,ativo,IBRX,pl12m,pl3m,plex,pvp,prec12m,prec3m,precex,...,evebitdaclsex,evebitemp12m,evebitemp3m,evebitempex,evebitcls12m,evebitcls3m,evebitclsex,evvd12m,evvd3m,evvdex
0,2001-12-28,ABEV3,1,19.011496255,71.159148628,27.395286977,4.9202690272,2.7174749515,11.003757519,3.7940575118,...,NaN,15.380073498,53.717503462,22.222734989,14.354088884,50.150156496,20.758734693,3.0921401865,12.520873765,4.3171539431
1,2002-01-31,ABEV3,1,19.829340979,87.815471781,19.829340979,4.5930309473,2.3840722928,7.2952797548,2.3840722928,...,8.0253868658,12.693891317,32.383892861,12.693891317,11.822504347,30.415910875,11.822504347,2.7111237393,8.2960597242,2.7111237393
2,2002-02-28,ABEV3,1,20.325074023,90.010856449,20.325074023,4.7078566097,2.4436740424,7.477661572,2.4436740424,...,8.2018181704,13.049876243,33.292060217,13.049876243,12.08241205,31.084578807,12.08241205,2.7707254889,8.4784415415,2.7707254889
3,2002-03-28,ABEV3,1,19.829340979,87.815471781,19.829340979,4.5930309473,2.3840722928,7.2952797548,2.3840722928,...,8.0253868658,12.577997614,32.088231811,12.577997614,11.822504347,30.415910875,11.822504347,2.7111237393,8.2960597242,2.7111237393
4,2002-04-30,ABEV3,1,20.819232161,70.719990131,70.719990131,4.7783498177,2.378045866,9.482743897,9.482743897,...,8.0253868658,13.260429675,61.340553763,61.340553763,12.223676437,56.153723785,56.153723785,2.7242644357,10.863332083,10.863332083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77123,2022-03-31,RAIZ4,0,19.39910225,290.62796596,19.39910225,3.3595802075,0.31938182889,1.3604891674,0.31938182889,...,6.2615923617,15.850783307,130.81929931,15.850783307,13.313628227,130.90567734,13.313628227,0.4294974284,1.8295549274,0.4294974284
77124,2022-04-29,RAIZ4,0,19.04037063,285.25362235,19.04037063,3.2974542577,0.31347576378,1.3353307618,0.31347576378,...,6.1754885349,15.632817489,129.02038912,15.632817489,13.130551101,129.10557936,13.130551101,0.42359136329,1.8043965218,0.42359136329
77125,2022-05-31,RAIZ4,1,16.915575646,253.42097174,16.915575646,2.9294774782,0.27849368579,1.1863155898,0.27849368579,...,5.6654889451,14.341789182,118.36530569,14.341789182,12.046171195,118.44346054,12.046171195,0.3886092853,1.6553813498,0.3886092853
77126,2022-06-30,RAIZ4,1,16.171269915,86.038645791,86.038645791,2.1220960878,0.20987284783,0.71666301941,0.71666301941,...,20.718657715,13.331050082,52.040068496,52.040068496,13.209444191,52.20522674,52.20522674,0.36681192295,1.252570511,1.252570511


In [37]:
#mercado1.dtypes

In [38]:
#Formatando variáveis
mercado1['ativo'] = mercado1['ativo'].astype(str)
mercado1.iloc[:,3:] = mercado1.iloc[:,3:].astype(float)
#mercado1.dtypes

In [39]:
#Base de dados com participantes do IBRX 
#mercado2=mercado1[mercado1['IBRX']==1]
mercado2=mercado1

In [40]:
mercado2=mercado2.sort_values(['data', "ativo"], ascending=[True, True])
#mercado2

In [42]:
mercado2=mercado2[['ativo','data',\
                   "IBRX",\
                "pl12m",\
                "pvp",\
                "prec12m",\
                "pebitda12m",\
                #"pfcf12m",\
                "lp12m",\
                "marketcap",\
                "ev",\
                "estrutcapvm",\
                "evebitdaemp12m",\
                "evebitemp12m",\
                 "evvd12m",]]
mercado2

,ativo,data,IBRX,pl12m,pvp,prec12m,pebitda12m,lp12m,marketcap,ev,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,1,19.011496,4.920269,2.717475,NaN,5.259975,1.793239e+07,2.021964e+07,17.667309,NaN,15.380073,3.092140
496,ABYA3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
744,ACES3,2001-12-28,1,-33.200975,0.362710,0.213802,0.883716,-3.011960,4.928269e+05,2.923945e+06,81.954876,5.701621,9.628597,1.360724
992,ACES4,2001-12-28,1,-37.555203,0.410279,0.241841,0.999613,-2.662747,4.928269e+05,2.923945e+06,81.954876,5.701621,9.628597,1.388763
1240,ACGU3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72415,VVAX11,2022-07-29,0,-15.753849,14.498341,5.948269,13.720889,-6.347655,2.097562e+06,2.212898e+06,9.639919,14.885938,-53.132080,6.275339
72663,WEGE3,2022-07-29,1,33.200040,8.771118,4.397995,24.725126,3.012045,1.176553e+08,1.178235e+08,2.368143,24.760362,27.925428,4.404282
72911,WEGE4,2022-07-29,0,18.228799,5.869202,2.975560,15.096831,5.485825,1.176553e+08,1.178235e+08,2.368143,24.760362,27.925428,2.879065
73159,WIZS3,2022-07-29,1,9.426188,3.100363,1.426353,3.709998,10.608742,1.186512e+06,1.725718e+06,16.428456,4.426735,5.652328,2.074554


In [43]:
#Outra análise visual de missing
#mercado2=mercado2.sort_values(['data', "ativo"], ascending=[True, True])
#balanco2=balanco2.sort_values(['data'], ascending=[True])
#sns.heatmap(mercado2.isnull(), cbar=False)
#sns.heatmap(price2.isnull(), cbar=False,yticklabels=list(price2['ativo']))

In [44]:
#mercado2.to_excel("mercado2.xlsx")

### Missing data

In [45]:
#Métricas para análise explanatória inicial
def estats(df):
    a=list(df.count()) #nonnullvalues
    
    b=list(df.isnull().sum() * 100 / len(df)) #percent_missing
   
    c = []#firstnonnullline
    for i in df.columns:
        y = df[i].first_valid_index()
        c.append(y) 
   
    d = [] #date_firstnonnullline
    for i in c:
        try:
            y = df.loc[i,'data']
            d.append(y)
        except Exception:
            y = np.nan
            d.append(y)

    e = [] #uniquevalues
    for i in df.columns:
        y = df[i].nunique()
        e.append(y)

    #f= list(map(lambda x,y: (x/y)*100, e,a)) #percentunique

    g = [] #zerocount
    for i in df.columns:
        y = (df[i] == "0").sum()
        g.append(y)
    
    h= list(map(lambda x,y: (x/y)*100, g,a)) #percentzero   
    
    #Selecionando algumas métricas mais relevantes
    estats=pd.DataFrame(data={'percent_missing': b, 'date_firstnonnullline': d, 'percentzero':h },index=df.columns)
    
    return estats.iloc[3:,:] #mostrando as métricas somente para as variáveis de fato

In [46]:
mercado2.dtypes

ativo                     object
data              datetime64[ns]
IBRX                       int64
pl12m                    float64
pvp                      float64
prec12m                  float64
pebitda12m               float64
lp12m                    float64
marketcap                float64
ev                       float64
estrutcapvm              float64
evebitdaemp12m           float64
evebitemp12m             float64
evvd12m                  float64
dtype: object

In [47]:
#Mostrando as métricas para toda base de dados
estats(mercado2)

,percent_missing,date_firstnonnullline,percentzero
pl12m,23.947205,2001-12-28,0.0
pvp,22.944975,2001-12-28,0.0
prec12m,29.107458,2001-12-28,0.0
pebitda12m,29.404367,2001-12-28,0.0
lp12m,23.947205,2001-12-28,0.0
...,...,...,...
ev,25.499170,2001-12-28,0.0
estrutcapvm,25.499170,2001-12-28,0.0
evebitdaemp12m,26.863137,2001-12-28,0.0
evebitemp12m,26.462504,2001-12-28,0.0


In [48]:
#Olhando para um ticher em especial
#mercado2[mercado2['ativo']=='ABEV3']

In [49]:
#Olhando para uma data em especial
#mercado2[mercado2['data']=='2001-12-28']

In [50]:
#Mostrando as métricas para um ticker em especial
#estats(mercado2[mercado2['ativo']=='ABEV3'])

In [51]:
#Mostrando as métricas para todos os tickers
#todos= list(map(lambda x: estats(mercado2[mercado2['ativo']==x]), list(mercado2['ativo'].unique())))
#estatsall=pd.concat(todos, axis=0, keys=list(mercado2['ativo'].unique()))
#estatsall

In [52]:
#Análise visual de missing
#df=mercado2.copy()
#df = df.sort_values(by=['data'])

#fontsize = 20
    
#fig, ax = plt.subplots(1, 1, figsize=[5.5, 10]) #mude aqui o tamanho da matriz
#msno.matrix(df=df.iloc[:,3:], ax=ax, color=(0.2, 0.2, 0.2), sparkline=False, fontsize=fontsize)

#label = list(df['ativo'].unique())
#ylim_start, ylim_end = ax.get_ylim()
#step_size = df.shape[0] / len(label)
#_ = ax.yaxis.set_ticks(np.arange(ylim_end, ylim_start, step_size))
#_ = ax.yaxis.set_ticklabels(label, fontsize=fontsize)

In [53]:
mercado31=mercado2[['ativo','data',\
                   "IBRX",\
                "marketcap",\
                "ev",]]
#mercado31

In [60]:
mercado32=mercado2[['ativo','data',\
                "pl12m",\
                "pvp",\
                "prec12m",\
                "pebitda12m",\
                #"pfcf12m",\
                "lp12m",\
                "estrutcapvm",\
                "evebitdaemp12m",\
                "evebitemp12m",\
                 "evvd12m",]]
#mercado32

In [61]:
mercado4=mercado32 ## Rodar apenas uma vez
mercado4 = mercado4.set_index('data', append=False) # Data como índice para possibilitar shift
mercado4 = mercado4.set_index('ativo', append=True)
#balanco4 = balanco4.groupby(level=1).ffill()
mercado4 = mercado4.groupby(level=1).shift(3) # Shift para defasagem de 3 meses para dados balanço
mercado4 = mercado4.reset_index(level=0) # Reset índice - 2 vezes level 0 para manter ordem das colunas
mercado4 = mercado4.reset_index(level=0)
mercado4.head()

,ativo,data,pl12m,pvp,prec12m,pebitda12m,lp12m,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABYA3,2001-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACES3,2001-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACES4,2001-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACGU3,2001-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
mercado4=mercado31.merge(mercado4, on=["data", "ativo"])
mercado4

,ativo,data,IBRX,marketcap,ev,pl12m,pvp,prec12m,pebitda12m,lp12m,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,1,1.793239e+07,2.021964e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABYA3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACES3,2001-12-28,1,4.928269e+05,2.923945e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACES4,2001-12-28,1,4.928269e+05,2.923945e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACGU3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77123,VVAX11,2022-07-29,0,2.097562e+06,2.212898e+06,-15.753849,14.498341,5.948269,13.720889,-6.347655,9.639919,14.885938,-53.132080,6.275339
77124,WEGE3,2022-07-29,1,1.176553e+08,1.178235e+08,33.540233,10.171132,4.989173,25.803652,2.981494,2.379601,25.841609,28.985288,4.996430
77125,WEGE4,2022-07-29,0,1.176553e+08,1.178235e+08,18.228799,5.869202,2.975560,15.096831,5.485825,2.379601,25.841609,28.985288,2.879065
77126,WIZS3,2022-07-29,1,1.186512e+06,1.725718e+06,7.397993,3.421781,1.484683,3.709998,13.517179,15.810796,4.426735,5.393196,2.175602


In [64]:
mercado2[mercado2["ativo"]=="ABEV3"].head(10)

,ativo,data,IBRX,pl12m,pvp,prec12m,pebitda12m,lp12m,marketcap,ev,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,1,19.011496,4.920269,2.717475,NaN,5.259975,1.793239e+07,2.021964e+07,17.667309,NaN,15.380073,3.092140
1,ABEV3,2002-01-31,1,19.829341,4.593031,2.384072,7.057259,5.043032,1.687641e+07,1.899565e+07,21.306513,8.616904,12.693891,2.711124
2,ABEV3,2002-02-28,1,20.325074,4.707857,2.443674,7.233690,4.920031,1.740912e+07,1.952836e+07,20.790089,8.858555,13.049876,2.770725
3,ABEV3,2002-03-28,1,19.829341,4.593031,2.384072,7.057259,5.043032,1.670298e+07,1.882222e+07,21.480219,8.538233,12.577998,2.711124
4,ABEV3,2002-04-30,1,20.819232,4.778350,2.378046,7.057259,4.803251,1.744488e+07,1.971669e+07,20.603483,8.538233,13.260430,2.724264
5,ABEV3,2002-05-31,1,20.615123,4.731504,2.354732,7.057259,4.850808,1.708639e+07,1.935820e+07,20.945226,8.538233,13.019325,2.700950
6,ABEV3,2002-06-28,1,19.186351,4.403577,2.191532,7.057259,5.212039,1.592512e+07,1.819693e+07,22.134494,8.538233,12.238317,2.537751
7,ABEV3,2002-07-31,1,14.756756,4.065092,2.135484,7.057259,6.776557,1.557270e+07,1.756134e+07,24.573377,8.538233,10.748536,2.426928
8,ABEV3,2002-08-30,1,15.225224,4.194143,2.203278,7.057259,6.568048,1.585322e+07,1.784186e+07,24.243974,8.538233,10.920229,2.494721
9,ABEV3,2002-09-30,1,11.823932,3.638675,2.111993,7.057259,8.457423,1.559080e+07,1.750013e+07,26.509462,8.538233,9.865444,2.384766


In [65]:
mercado4[mercado4["ativo"]=="ABEV3"].head(10)

,ativo,data,IBRX,marketcap,ev,pl12m,pvp,prec12m,pebitda12m,lp12m,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,1,1.793239e+07,2.021964e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,ABEV3,2002-01-31,1,1.687641e+07,1.899565e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622,ABEV3,2002-02-28,1,1.740912e+07,1.952836e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,ABEV3,2002-03-28,1,1.670298e+07,1.882222e+07,19.011496,4.920269,2.717475,NaN,5.259975,17.667309,NaN,15.380073,3.092140
1244,ABEV3,2002-04-30,1,1.744488e+07,1.971669e+07,19.829341,4.593031,2.384072,7.057259,5.043032,21.306513,8.616904,12.693891,2.711124
1555,ABEV3,2002-05-31,1,1.708639e+07,1.935820e+07,20.325074,4.707857,2.443674,7.233690,4.920031,20.790089,8.858555,13.049876,2.770725
1866,ABEV3,2002-06-28,1,1.592512e+07,1.819693e+07,19.829341,4.593031,2.384072,7.057259,5.043032,21.480219,8.538233,12.577998,2.711124
2177,ABEV3,2002-07-31,1,1.557270e+07,1.756134e+07,20.819232,4.778350,2.378046,7.057259,4.803251,20.603483,8.538233,13.260430,2.724264
2488,ABEV3,2002-08-30,1,1.585322e+07,1.784186e+07,20.615123,4.731504,2.354732,7.057259,4.850808,20.945226,8.538233,13.019325,2.700950
2799,ABEV3,2002-09-30,1,1.559080e+07,1.750013e+07,19.186351,4.403577,2.191532,7.057259,5.212039,22.134494,8.538233,12.238317,2.537751


In [66]:
#Análise visual de missing
#df=mercado4.copy()
#df = df.sort_values(by=['data'])
#fontsize = 12    
#fig, ax = plt.subplots(1, 1, figsize=[15, 5.5]) #mude aqui o tamanho da matriz
#msno.matrix(df=df.iloc[:,3:], ax=ax, color=(0.2, 0.2, 0.2), sparkline=False, fontsize=fontsize)

In [67]:
mercado4

,ativo,data,IBRX,marketcap,ev,pl12m,pvp,prec12m,pebitda12m,lp12m,estrutcapvm,evebitdaemp12m,evebitemp12m,evvd12m
0,ABEV3,2001-12-28,1,1.793239e+07,2.021964e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABYA3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACES3,2001-12-28,1,4.928269e+05,2.923945e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACES4,2001-12-28,1,4.928269e+05,2.923945e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACGU3,2001-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77123,VVAX11,2022-07-29,0,2.097562e+06,2.212898e+06,-15.753849,14.498341,5.948269,13.720889,-6.347655,9.639919,14.885938,-53.132080,6.275339
77124,WEGE3,2022-07-29,1,1.176553e+08,1.178235e+08,33.540233,10.171132,4.989173,25.803652,2.981494,2.379601,25.841609,28.985288,4.996430
77125,WEGE4,2022-07-29,0,1.176553e+08,1.178235e+08,18.228799,5.869202,2.975560,15.096831,5.485825,2.379601,25.841609,28.985288,2.879065
77126,WIZS3,2022-07-29,1,1.186512e+06,1.725718e+06,7.397993,3.421781,1.484683,3.709998,13.517179,15.810796,4.426735,5.393196,2.175602


In [41]:
mercado4.to_excel("mer.xlsx")